# Datos de reportes mineros consolidados y guardados en base de datos

Se nos presentan los datos de reportes de producción minera en archivos csv, los cuales debemos trabajar mediante ETL para cargarlos a una base de datos

## Importamos las librerías

In [1]:
import pandas as pd
from numpy import nan
import sqlalchemy
import psycopg2

## 1. Extract

Realizamos la extracción de datos desde los archivos csv.
También deberemos analizar la composición de los datos obtenidos.

In [2]:
df_datos = pd.read_csv(filepath_or_buffer="DatosEjemploDiciembre2.csv", sep=";",encoding='latin-1')

In [3]:
df_datos

,Fecha,Carguio,Cami¢n,Flota,Material,Origen,Zona,Destino,Tonelaje,Ciclos,Rajo
0,2022/12/01,CF2,C11,CAT 797 B,Sulfuro de Media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,"381,6530151",1,ESPERANZA
1,2022/12/01,CF2,C132,CAT 793 C,Sulfuro de Media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,226,1,ESPERANZA
2,2022/12/01,CF2,C160,CAT 793 F,Sulfuro de Media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,452,2,ESPERANZA
3,2022/12/01,CF2,C39,CAT 797 F,Sulfuro de Media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,"381,6530151",1,ESPERANZA
4,2022/12/01,CF2,C51,KOM 930 E,Sulfuro de Media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,"314,5620117",1,ESPERANZA
...,...,...,...,...,...,...,...,...,...,...,...
16631,2022/12/31,PA9,C47,CAT 797 F,Sulfuro de Alta Ley,ESP/F08/1800/803,FASE 8,STOCK_SAL_DIN,"348,0710144",1,ESPERANZA
16632,2022/12/31,PA9,C49,KOM 930 E,Sulfuro de Alta Ley,ESP/F08/1800/803,FASE 8,STOCK_SAL_DIN,"288,8269958",1,ESPERANZA
16633,2022/12/31,PA9,C50,KOM 930 E,Lastre Sulfurado,ESP/F08/1832/824,FASE 8,BOT_RAMPA_SUPERIOR,"866,4810181",3,ESPERANZA
16634,2022/12/31,PA9,C55,KOM 930 E,Sulfuro de Alta Ley,ESP/F08/1800/803,FASE 8,CHANCADO-SULFURO,"288,8269958",1,ESPERANZA


## 2. Transform

Separamos los datos por columnnas

In [4]:
colsDatos = ['fecha', 'carguio', 'camion', 'flota', 'material', 'origen', 'zona', 'destino', 'tonelaje', 'ciclos', 'rajo']
df_datos.columns = colsDatos

Obtenemos los tipos de datos para el dataframe

In [5]:
df_datos.dtypes

fecha       object
carguio     object
camion      object
flota       object
material    object
origen      object
zona        object
destino     object
tonelaje    object
ciclos       int64
rajo        object
dtype: object

Buscamos la cantidad de datos en el dataframe

In [6]:
df_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fecha     16636 non-null  object
 1   carguio   16636 non-null  object
 2   camion    16636 non-null  object
 3   flota     16636 non-null  object
 4   material  16636 non-null  object
 5   origen    16636 non-null  object
 6   zona      16636 non-null  object
 7   destino   16636 non-null  object
 8   tonelaje  16636 non-null  object
 9   ciclos    16636 non-null  int64 
 10  rajo      16636 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.4+ MB


Sumamos los datos nulos

In [7]:
df_datos.isnull().sum()

fecha       0
carguio     0
camion      0
flota       0
material    0
origen      0
zona        0
destino     0
tonelaje    0
ciclos      0
rajo        0
dtype: int64

Al no haber datos nulos en el dataframe se continua, de haber nulos se procederá a eliminarlos

### Formateo de datos

La mezcla de mayúsculas y minúsculas (camel case) suele dar problemas, por lo que decidimos pasar los valores de la columna material a minúscula

In [9]:
df_datos["material"] = df_datos["material"].str.lower()
df_datos

,fecha,carguio,camion,flota,material,origen,zona,destino,tonelaje,ciclos,rajo
0,2022/12/01,CF2,C11,CAT 797 B,sulfuro de media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,"381,6530151",1,ESPERANZA
1,2022/12/01,CF2,C132,CAT 793 C,sulfuro de media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,226,1,ESPERANZA
2,2022/12/01,CF2,C160,CAT 793 F,sulfuro de media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,452,2,ESPERANZA
3,2022/12/01,CF2,C39,CAT 797 F,sulfuro de media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,"381,6530151",1,ESPERANZA
4,2022/12/01,CF2,C51,KOM 930 E,sulfuro de media,STOCK_SME,ESPERANZA,CHANCADO-SULFURO,"314,5620117",1,ESPERANZA
...,...,...,...,...,...,...,...,...,...,...,...
16631,2022/12/31,PA9,C47,CAT 797 F,sulfuro de alta ley,ESP/F08/1800/803,FASE 8,STOCK_SAL_DIN,"348,0710144",1,ESPERANZA
16632,2022/12/31,PA9,C49,KOM 930 E,sulfuro de alta ley,ESP/F08/1800/803,FASE 8,STOCK_SAL_DIN,"288,8269958",1,ESPERANZA
16633,2022/12/31,PA9,C50,KOM 930 E,lastre sulfurado,ESP/F08/1832/824,FASE 8,BOT_RAMPA_SUPERIOR,"866,4810181",3,ESPERANZA
16634,2022/12/31,PA9,C55,KOM 930 E,sulfuro de alta ley,ESP/F08/1800/803,FASE 8,CHANCADO-SULFURO,"288,8269958",1,ESPERANZA


Notamos que podemos cambiar el tipo de tonelaje desde objtect a float, para ello primero debemos reemplazar las "comas" por "puntos"

In [11]:
df_datos['tonelaje'] = df_datos['tonelaje'].str.replace(',', '.') #separamos correctamente los datos
df_datos = df_datos.astype({'tonelaje': 'float64'})
df_datos.dtypes

fecha        object
carguio      object
camion       object
flota        object
material     object
origen       object
zona         object
destino      object
tonelaje    float64
ciclos        int64
rajo         object
dtype: object

Notamos que también podemos cambiar el formato a las fechas

In [13]:
df_datos['fecha'] = df_datos['fecha'].str.replace('/', '-') #separamos correctamente los datos
df_datos['fecha'] = pd.to_datetime(df_datos['fecha'])
df_datos.dtypes

fecha       datetime64[ns]
carguio             object
camion              object
flota               object
material            object
origen              object
zona                object
destino             object
tonelaje           float64
ciclos               int64
rajo                object
dtype: object

## 3. Load

A continuación cargamos nuestros dataframes a una base de datos en PostgreSQL